In [ ]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.AI.HuggingFace, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Qdrant, *-*"

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.Memory.Qdrant;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.AI.HuggingFace.TextEmbedding;

In [ ]:
string chat_endpoint = "http://localhost:5002/v1/chat/completions";
string embeddings_endpoint = "http://localhost:5002/v1/embeddings";
string qdrant_endpoint = "http://localhost:6333";

In [ ]:

#pragma warning disable SKEXP0020

Kernel kernel = new KernelBuilder()
            .AddHuggingFaceTextGeneration(
                model: "phi15",
                endpoint: chat_endpoint)
            .Build();

In [ ]:
#pragma warning disable SKEXP0052
#pragma warning disable CS1061
#pragma warning disable SKEXP0011
#pragma warning disable SKEXP0026

#pragma warning disable SKEXP0020

var qdrantMemoryBuilder = new MemoryBuilder();

var hfembeddings = new HuggingFaceTextEmbeddingGeneration("jina", embeddings_endpoint);

qdrantMemoryBuilder.WithTextEmbeddingGeneration(hfembeddings);
qdrantMemoryBuilder.WithQdrantMemoryStore(qdrant_endpoint, 768);

var builder = qdrantMemoryBuilder.Build();

In [ ]:
string MemoryCollectionName = "jina_vb";

await builder.SaveInformationAsync(MemoryCollectionName, id: "id1", text: "kinfey is cloud advocate");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id2", text: "kinfey is Microsoft MVP");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id3", text: "kinfey is AI expert");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id4", text: "Satya is Microsoft CEO");


In [ ]:
var searchResults =  builder.SearchAsync(MemoryCollectionName, "who is kinfey", limit: 3, minRelevanceScore: 0.6);

In [ ]:
await foreach (var item in searchResults)
{
    Console.WriteLine(item.Metadata.Text + " : " + item.Relevance);
}

In [ ]:
var questionAnswerFunction = kernel.CreateFunctionFromPrompt("ask: {{$input}} answer:");

In [ ]:
var result = await kernel.InvokeAsync(questionAnswerFunction, new("who are you"));


In [ ]:
result.GetValue<string>()